In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard

In [2]:
# Configuration
data_dir = "../../datasets/data21k"
batch_size = 32
img_size = (224, 224)
num_classes = 5 # Update based on your dataset
learning_rate = 0.001
num_epochs = 20

In [3]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,  # 20% for validation
    subset="training",
    seed=123,  # Random seed for reproducibility
    image_size=img_size,
    batch_size=batch_size,
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
)

Found 21397 files belonging to 5 classes.
Using 17118 files for training.
Found 21397 files belonging to 5 classes.
Using 4279 files for validation.


In [5]:
def normalize(image, label):
    image = tf.image.per_image_standardization(image)  # Normalize by std deviation
    return image, label

train_dataset = train_dataset.map(normalize)
val_dataset = val_dataset.map(normalize)

train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

class_names = ['CBB', 'CBSD', 'CGM', 'CH', 'CMD']

print("Class Names:", class_names)

for image_batch, label_batch in train_dataset.take(1):
    print(f"Image batch shape: {image_batch.shape}")
    print(f"Label batch shape: {label_batch.shape}")

Class Names: ['CBB', 'CBSD', 'CGM', 'CH', 'CMD']
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32,)


In [6]:
# Model Definition
base_model = EfficientNetV2S(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)


82420632/82420632 [==============================] - 6s 0us/step


In [7]:
# Define TensorBoard callback
log_dir = "./logs"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Compile the Model
model.compile(
   optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
   loss="categorical_crossentropy",
   metrics=["accuracy"]
)

# Training
model.fit(
   train_dataset,
   validation_data=val_dataset,
   epochs=num_epochs,
   callbacks=[tensorboard_callback],
   verbose=1  # 0 = silent, 1 = progress bar, 2 = one line per epoch
)

Epoch 1/20


ValueError: in user code:

    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\nicho\anaconda3\envs\cassava\lib\site-packages\keras\engine\compile_utils.py", line 265, in 

In [ ]:
# Save the Model
model.save("efficientnetv2_model.h5")